In [18]:
import pandas as pd
import os
import altair as alt
import numpy as np
import plotnine as pt


In [2]:
os.chdir('/Users/trishapunamiya/Desktop/LSE/Data Viz/Project/Raw Data')

In [3]:
# Load enrollment data
df_enr = pd.read_csv('2024/100_enr1.csv')

# Load school profile data (has state, district, rural_urban info)
# Try 100_prof2.csv or the School Basic Profile file
df_profile = pd.read_csv('2024/100_prof1.csv')  # Use actual filename

# Merge on pseudocode (school ID)
df_merged = df_enr.merge(df_profile[['pseudocode', 'state', 'district', 'rural_urban']], 
                          on='pseudocode', 
                          how='left')

Gender Gap by State

In [4]:
# Class columns for boys and girls
boy_cols = ['cpp_b', 'c1_b', 'c2_b', 'c3_b', 'c4_b', 'c5_b', 'c6_b', 
            'c7_b', 'c8_b', 'c9_b', 'c10_b', 'c11_b', 'c12_b']
girl_cols = ['cpp_g', 'c1_g', 'c2_g', 'c3_g', 'c4_g', 'c5_g', 'c6_g', 
             'c7_g', 'c8_g', 'c9_g', 'c10_g', 'c11_g', 'c12_g']


# Calculate total boys and girls for each row
df_merged['total_boys'] = df_merged[boy_cols].sum(axis=1)
df_merged['total_girls'] = df_merged[girl_cols].sum(axis=1)
df_merged['total_students'] = df_merged['total_boys'] + df_merged['total_girls']


# Group by state and sum
state_gender = df_merged.groupby('state').agg({
    'total_boys': 'sum',
    'total_girls': 'sum'
}).reset_index()

# Calculate gender gap
state_gender['total_enrollment'] = state_gender['total_boys'] + state_gender['total_girls']
state_gender['girls_percentage'] = (state_gender['total_girls'] / state_gender['total_enrollment']) * 100
state_gender['boys_percentage'] = (state_gender['total_boys'] / state_gender['total_enrollment']) * 100

# Gender gap: positive means more girls, negative means more boys
# Gender Parity Index (GPI) - UNESCO standard
state_gender['gender_gap'] = state_gender['girls_percentage'] - state_gender['boys_percentage']

# Sort by gender gap
state_gender = state_gender.sort_values('gender_gap').reset_index(drop=True)

In [5]:
state_gender.to_json('gender_gap_by_state.json', orient='records', indent=2)

# Bubble Chart - Student/Teacher

In [7]:
# Load teacher data
df_teacher = pd.read_csv('2024/100_tch.csv')

# Merge teacher data with state info on pseudocode
df_teacher_merged = df_teacher.merge(
    df_profile[['pseudocode', 'state']], 
    on='pseudocode', 
    how='left'
)

# Aggregate teachers by state
state_teachers = df_teacher_merged.groupby('state').agg({
    'total_tch': 'sum'
}).reset_index()

# Aggregate schools and students by state
schools_per_state = df_merged.groupby('state').agg({
    'pseudocode': 'nunique',  # Count unique schools
    'total_students': 'sum'
}).reset_index()

# Rename pseudocode to num_schools (DO THIS BEFORE MERGING)
schools_per_state.rename(columns={'pseudocode': 'num_schools'}, inplace=True)

# Calculate average students per school
schools_per_state['avg_students_per_school'] = schools_per_state['total_students'] / schools_per_state['num_schools']

# Merge enrollment and teacher data
state_data = schools_per_state.merge(state_teachers, on='state')

# Calculate student-teacher ratio
state_data['student_teacher_ratio'] = state_data['total_students'] / state_data['total_tch']

# Add region classification
def classify_region(state):
    """Classify Indian states into regions"""
    north = ['JAMMU & KASHMIR', 'HIMACHAL PRADESH', 'PUNJAB', 'CHANDIGARH', 
             'UTTARAKHAND', 'HARYANA', 'DELHI', 'RAJASTHAN']
    east_central = ['UTTAR PRADESH', 'BIHAR', 'JHARKHAND', 'WEST BENGAL', 
                    'ODISHA', 'CHHATTISGARH']
    west = ['MADHYA PRADESH', 'GUJARAT', 'MAHARASHTRA', 'GOA', 
            'DADRA & NAGAR HAVELI AND DAMAN & DIU', 'DAMAN & DIU', 
            'DADRA AND NAGAR HAVELI']
    south = ['ANDHRA PRADESH', 'KARNATAKA', 'KERALA', 'TAMIL NADU', 
             'TELANGANA', 'PUDUCHERRY', 'LAKSHADWEEP', 
             'ANDAMAN & NICOBAR ISLANDS', 'ANDAMAN AND NICOBAR ISLANDS']
    northeast = ['ARUNACHAL PRADESH', 'ASSAM', 'MANIPUR', 'MEGHALAYA', 
                 'MIZORAM', 'NAGALAND', 'SIKKIM', 'TRIPURA']
    
    state_upper = state.upper()
    if state_upper in north:
        return 'North'
    elif state_upper in east_central:
        return 'East/Central'
    elif state_upper in west:
        return 'West'
    elif state_upper in south:
        return 'South'
    elif state_upper in northeast:
        return 'Northeast'
    else:
        return 'Other'

state_data['region'] = state_data['state'].apply(classify_region)

print(state_data[['state', 'num_schools', 'avg_students_per_school', 'student_teacher_ratio', 'region']])

# Save to JSON
state_data.to_json('student_teacher_bubble.json', orient='records', indent=2)

print("\n✓ Data saved!")
print(f"\nAvg students per school range: {state_data['avg_students_per_school'].min():.0f} to {state_data['avg_students_per_school'].max():.0f}")

                                   state  num_schools  \
0              ANDAMAN & NICOBAR ISLANDS          408   
1                         ANDHRA PRADESH        61316   
2                      ARUNACHAL PRADESH         3208   
3                                  ASSAM        55283   
4                                  BIHAR        94334   
5                             CHANDIGARH          207   
6                           CHHATTISGARH        56802   
7   DADRA & NAGAR HAVELI AND DAMAN & DIU          433   
8                                  DELHI         5556   
9                                    GOA         1479   
10                               GUJARAT        53292   
11                               HARYANA        23494   
12                      HIMACHAL PRADESH        17330   
13                       JAMMU & KASHMIR        24046   
14                             JHARKHAND        44269   
15                             KARNATAKA        74589   
16                             

In [8]:
import pandas as pd
import altair as alt

# Assuming df_merged has enrollment data

# Define class columns
class_levels = ['cpp', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12']
class_labels = ['Pre-Primary', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 
                'Class 6', 'Class 7', 'Class 8', 'Class 9', 'Class 10', 'Class 11', 'Class 12']

# Aggregate by class level
dropout_data = []

for i, class_level in enumerate(class_levels):
    boys_col = f'{class_level}_b'
    girls_col = f'{class_level}_g'
    
    total_boys = df_merged[boys_col].sum()
    total_girls = df_merged[girls_col].sum()
    
    dropout_data.append({
        'Class': class_labels[i],
        'Gender': 'Boys',
        'Enrollment': total_boys,
        'Class_Order': i
    })
    
    dropout_data.append({
        'Class': class_labels[i],
        'Gender': 'Girls',
        'Enrollment': total_girls,
        'Class_Order': i
    })

df_dropout = pd.DataFrame(dropout_data)

# Create the chart with Altair
chart = alt.Chart(df_dropout).mark_line(point=True, strokeWidth=3).encode(
    x=alt.X('Class:N', 
            axis=alt.Axis(title=None, labelAngle=-45),
            sort=class_labels),
    y=alt.Y('Enrollment:Q',
            axis=alt.Axis(title='Number of Students Enrolled', format='~s')),
    color=alt.Color('Gender:N',
                    scale=alt.Scale(domain=['Boys', 'Girls'], 
                                   range=['#5499C7', '#E74C3C']),
                    legend=alt.Legend(title='Gender')),
    tooltip=['Class', 'Gender', alt.Tooltip('Enrollment:Q', format=',')]
).properties(
    width=650,
    height=400,
    title={
        "text": "Student Enrollment Drop-off by Class Level",
        "subtitle": "All-India enrollment from Pre-Primary through Class 12 | Data: UDISE+ India",
        "fontSize": 16,
        "subtitleFontSize": 12
    }
)

# Display the chart
chart.show()

# Save as JSON for Vega-Lite

alt.Chart(...)

In [ ]:
# List of years to process
years = [2022, 2023]

# Store all data
all_years_data = []

for year in years:
    print(f"Processing {year}...")
    
    # Load enrollment data for this year
    enr_file = f'/Users/trishapunamiya/Desktop/LSE/Data Viz/Project/Raw Data/{year}/100_enr1.csv'
    df_enr = pd.read_csv(enr_file)
    
    # Load profile data for this year to get state info
    profile_file = f'/Users/trishapunamiya/Desktop/LSE/Data Viz/Project/Raw Data/{year}/100_prof1.csv'  # Adjust filename if different
    df_profile = pd.read_csv(profile_file)
    
    # Merge
    df_merged_year = df_enr.merge(df_profile[['pseudocode', 'state']], on='pseudocode', how='left')
    
    # Add year column
    df_merged_year['year'] = year
    
    # Append to list
    all_years_data.append(df_merged_year)
    
    print(f"  ✓ Loaded {len(df_merged_year):,} rows for {year}")

# Combine all years
df_all_years = pd.concat(all_years_data, ignore_index=True)\\\\\\\\

Processing 2022...
  ✓ Loaded 7,695,344 rows for 2022
Processing 2023...
  ✓ Loaded 8,234,734 rows for 2023


### Loading 2021-2024 Enrolment and School Profile Data

In [22]:
years = [2022, 2023, 2024]
all_years_data = []

for year in years:
    print(f"\n=== Processing {year} ===")
    
    # Load enrollment data for this year
    enr_file = f'/Users/trishapunamiya/Desktop/LSE/Data Viz/Project/Raw Data/{year}/100_enr1.csv'  # Adjust path as needed
    df_enr = pd.read_csv(enr_file)
    print(f"Loaded enrollment: {len(df_enr):,} rows")
    
    # Load profile data for this year
    profile_file = f'/Users/trishapunamiya/Desktop/LSE/Data Viz/Project/Raw Data/{year}/100_prof1.csv'
    df_profile = pd.read_csv(profile_file)
    print(f"Loaded profile: {len(df_profile):,} rows")
    
    # Merge
    df_merged_year = df_enr.merge(df_profile[['pseudocode', 'state']], on='pseudocode', how='left')
    print(f"After merge: {len(df_merged_year):,} rows")
    
    # Add year column
    df_merged_year['year'] = year
    
    # Append to list
    all_years_data.append(df_merged_year)
    print(f"✓ Added {year} to list (list now has {len(all_years_data)} dataframes)")

# 2021 data
profile_2021 = pd.read_csv('/Users/trishapunamiya/Desktop/LSE/Data Viz/Project/Raw Data/2021/nationalProfile_1.csv')
enroll_2021 = pd.read_csv('/Users/trishapunamiya/Desktop/LSE/Data Viz/Project/Raw Data/2021/22-100_enr1/nationalEnrol1.csv')
df_merged_2021 = enroll_2021.merge(profile_2021[['psuedocode', 'state']], on='psuedocode', how='left')
df_merged_2021['year'] = 2021
all_years_data.append(df_merged_2021)

# Combine all years
df_all = pd.concat(all_years_data, ignore_index=True)

print(f"\n=== FINAL RESULT ===")
print(f"Total rows: {len(df_all):,}")
print(f"Years in combined data: {df_all['year'].unique()}")


=== Processing 2022 ===
Loaded enrollment: 7,695,344 rows
Loaded profile: 1,466,109 rows
After merge: 7,695,344 rows
✓ Added 2022 to list (list now has 1 dataframes)

=== Processing 2023 ===
Loaded enrollment: 8,234,734 rows
Loaded profile: 1,471,891 rows
After merge: 8,234,734 rows
✓ Added 2023 to list (list now has 2 dataframes)

=== Processing 2024 ===
Loaded enrollment: 8,580,816 rows
Loaded profile: 1,471,473 rows
After merge: 8,580,816 rows
✓ Added 2024 to list (list now has 3 dataframes)

=== FINAL RESULT ===
Total rows: 29,268,860
Years in combined data: [2022 2023 2024 2021]


In [26]:
# Cleaning of the merged dataset
df_all['state'] = df_all['state'].astype(str).str.strip().str.title()

# changing Tamilnadu to Tamil Nadu
df_all['state'] = df_all['state'].replace('Tamilnadu', 'Tamil Nadu')     

In [89]:
# Check states with data in 2021
df_2021 = df_dropout[df_dropout['Year'] == 2021]

print(f"Total states in dataset: {df_dropout['State'].nunique()}")
print(f"States with data in 2021: {df_2021['State'].nunique()}")
print(f"\nTotal rows for 2021: {len(df_2021)}")

# See which states have data in 2021
states_2021 = sorted(df_2021['State'].unique())
print(f"\nStates with 2021 data:")
for state in states_2021:
    state_data = df_2021[df_2021['State'] == state]
    total_enrollment = state_data['Enrollment'].abs().sum()
    print(f"  {state}: {len(state_data)} rows, {total_enrollment:,.0f} total enrollment")

# Check which states are missing 2021 data
all_states = set(df_dropout['State'].unique())
states_with_2021 = set(df_2021['State'].unique())
missing_2021 = sorted(all_states - states_with_2021)

if missing_2021:
    print(f"\nStates MISSING 2021 data ({len(missing_2021)}):")
    for state in missing_2021:
        print(f"  {state}")

# Also check what years are available
print(f"\nAvailable years: {sorted(df_dropout['Year'].unique())}")

# print list of states with data in 2024
df_2024 = df_dropout[df_dropout['Year'] == 2024]
states_2024 = sorted(df_2024['State'].unique())

Total states in dataset: 37
States with data in 2021: 25

Total rows for 2021: 600

States with 2021 data:
  All India: 24 rows, 126,765,763 total enrollment
  Andaman & Nicobar Islands: 24 rows, 151,705 total enrollment
  Arunachal Pradesh: 24 rows, 733,159 total enrollment
  Chandigarh: 24 rows, 525,753 total enrollment
  Chhattisgarh: 24 rows, 14,387,235 total enrollment
  Dadra & Nagar Haveli And Daman & Diu: 24 rows, 261,430 total enrollment
  Delhi: 24 rows, 8,829,947 total enrollment
  Goa: 24 rows, 639,529 total enrollment
  Gujarat: 24 rows, 22,608,909 total enrollment
  Haryana: 24 rows, 11,098,194 total enrollment
  Himachal Pradesh: 24 rows, 2,450,679 total enrollment
  Jammu & Kashmir: 24 rows, 6,716,730 total enrollment
  Jharkhand: 24 rows, 17,403,082 total enrollment
  Kerala: 24 rows, 15,472,681 total enrollment
  Ladakh: 24 rows, 142,935 total enrollment
  Lakshadweep: 24 rows, 36,275 total enrollment
  Manipur: 24 rows, 1,461,657 total enrollment
  Meghalaya: 24 rows

In [107]:
# -------------------------------------------------------
# STEP 1 — Prepare funnel data for all years and states
# -------------------------------------------------------
class_levels = ['c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12']
class_labels = ['Class 1','Class 2','Class 3','Class 4','Class 5','Class 6',
                'Class 7','Class 8','Class 9','Class 10','Class 11','Class 12']

records = []

for year in sorted(df_all['year'].unique()):
    df_year = df_all[df_all['year'] == year]
    
    # Aggregate All-India
    for i, class_level in enumerate(class_levels):
        boys_col = f'{class_level}_b'
        girls_col = f'{class_level}_g'
        total_boys = df_year[boys_col].sum()
        total_girls = df_year[girls_col].sum()
        records.append({'Class': class_labels[i], 'Gender': 'Boys', 'Enrollment': -total_boys,
                        'Year': year, 'State': 'All India'})
        records.append({'Class': class_labels[i], 'Gender': 'Girls', 'Enrollment': total_girls,
                        'Year': year, 'State': 'All India'})

    # Aggregate by state
    for state in df_year['state'].unique():
        df_state = df_year[df_year['state'] == state]
        for i, class_level in enumerate(class_levels):
            boys_col = f'{class_level}_b'
            girls_col = f'{class_level}_g'
            total_boys = df_state[boys_col].sum()
            total_girls = df_state[girls_col].sum()
            records.append({'Class': class_labels[i], 'Gender': 'Boys', 'Enrollment': -total_boys,
                            'Year': year, 'State': state})
            records.append({'Class': class_labels[i], 'Gender': 'Girls', 'Enrollment': total_girls,
                            'Year': year, 'State': state})

df_dropout = pd.DataFrame(records)
df_dropout['Class'] = pd.Categorical(df_dropout['Class'], categories=class_labels[::-1], ordered=True)

# -------------------------------------------------------
# STEP 2 — Precompute max enrollment per state (RUN THIS FIRST)
# -------------------------------------------------------
state_max = df_dropout.groupby('State')['Enrollment'].apply(
    lambda x: max(abs(x.min()), abs(x.max()))
).reset_index()
state_max = state_max.rename(columns={'Enrollment': 'max_enrollment'})
df_dropout = df_dropout.merge(state_max, on='State', how='left')

# -------------------------------------------------------
# STEP 3 — Altair interactive funnel chart (MATCHED THEME)
# -------------------------------------------------------

year_param = alt.param(
    name='YearParam',
    value=int(df_dropout['Year'].min()),
    bind=alt.binding_range(
        min=int(df_dropout['Year'].min()),
        max=int(df_dropout['Year'].max()),
        step=1,
        name='Year: '
    )
)

state_param = alt.param(
    name='StateParam',
    value='All India',
    bind=alt.binding_select(
        options=sorted(df_dropout['State'].unique()),
        name='State: '
    )
)

# Create a dummy dataset for scale anchors
scale_anchors = []
for state in df_dropout['State'].unique():
    max_val = df_dropout[df_dropout['State'] == state]['max_enrollment'].iloc[0]
    scale_anchors.append({'State': state, 'anchor_value': -max_val})
    scale_anchors.append({'State': state, 'anchor_value': max_val})

df_anchors = pd.DataFrame(scale_anchors)

# Base layer - invisible points to set scale
base = alt.Chart(df_anchors).transform_filter(
    alt.datum.State == state_param
).mark_point(opacity=0).encode(
    x='anchor_value:Q'
).add_params(state_param)

# Main chart layer
bars = alt.Chart(df_dropout).transform_filter(
    alt.datum.State == state_param
).transform_filter(
    alt.datum.Year == year_param
).mark_bar().encode(
    y=alt.Y('Class:N', 
        sort=class_labels[::-1], 
        title=None,
        axis=alt.Axis(labelPadding=10, labelFontSize=12)
    ),
    x=alt.X('Enrollment:Q',
        axis=alt.Axis(
            title='Number of Students (thousands)',
            labelExpr="abs(datum.value)/1000",
            grid=False,
            tickCount=8,
            labelFontSize=12,
            titleFontSize=12
        )
    ),
    color=alt.Color('Gender:N', 
        scale=alt.Scale(domain=['Boys','Girls'], range=['#7B68A6','#D4936A']),
        legend=alt.Legend(
            orient='right',
            titleFontSize=12,
            labelFontSize=12,
            symbolSize=200
        )
    )
).add_params(year_param)

# Add a center line at zero
rule = alt.Chart(pd.DataFrame({'x': [0]})).mark_rule(
    color='gray',
    strokeWidth=1.5,
    strokeDash=[5, 5]
).encode(x='x:Q')

# Layer them together
chart = alt.layer(base, bars, rule).resolve_scale(
    x='shared'
).properties(
    width=700,
    height=450,
    title={
        'text': 'Education Pipeline: Enrollment by Class and Gender',
        'subtitle': 'Note: 2021 data only available for 25 of 37 states (including union territories)',
        'fontSize': 16,
        'subtitleFontSize': 12,
        'subtitleColor': '#666',
        'anchor': 'start'
    }
).configure_view(
    strokeWidth=1, stroke='gray'
).configure_axis(
    domainWidth=1
)

chart

alt.LayerChart(...)

In [106]:
chart.save('/Users/trishapunamiya/tpunamiya.github.io-4/education_dropout_byStateGender.json')

In [ ]:
# Create a line chart from 2021-2024 showing total enrollment over time
line_data = df_all.groupby(['year']).agg({
    'cpp_b': 'sum', 'cpp_g': 'sum',
    'c1_b': 'sum', 'c1_g': 'sum',
    'c2_b': 'sum', 'c2_g': 'sum',
    'c3_b': 'sum', 'c3_g': 'sum',
    'c4_b': 'sum', 'c4_g': 'sum',
    'c5_b': 'sum', 'c5_g': 'sum',
    'c6_b': 'sum', 'c6_g': 'sum',
    'c7_b': 'sum', 'c7_g': 'sum',
    'c8_b': 'sum', 'c8_g': 'sum',
    'c9_b': 'sum', 'c9_g': 'sum',
    'c10_b': 'sum', 'c10_g': 'sum',
    'c11_b': 'sum', 'c11_g': 'sum',
    'c12_b': 'sum', 'c12_g': 'sum'
}).reset_index()
line_data['
total_enrollment'] = line_data.sum(axis=1) - line_data['year']
line_data = line_data[['year', 'total_enrollment']]

In [109]:
# -------------------------------------------------------
# Create line chart for total enrollment over time
# -------------------------------------------------------

# Aggregate total enrollment by year and gender
enrollment_totals = []

for year in sorted(df_all['year'].unique()):
    df_year = df_all[df_all['year'] == year]
    
    # Sum all boys across all classes
    total_boys = sum([df_year[f'{class_level}_b'].sum() for class_level in class_levels])
    
    # Sum all girls across all classes
    total_girls = sum([df_year[f'{class_level}_g'].sum() for class_level in class_levels])
    
    enrollment_totals.append({
        'Year': year,
        'Gender': 'Boys',
        'Total Enrollment': total_boys
    })
    
    enrollment_totals.append({
        'Year': year,
        'Gender': 'Girls',
        'Total Enrollment': total_girls
    })

df_totals = pd.DataFrame(enrollment_totals)

# Create the line chart
line_chart = alt.Chart(df_totals).mark_line(
    point=True,
    strokeWidth=3
).encode(
    x=alt.X('Year:O',
        axis=alt.Axis(
            title='Year',
            labelAngle=0,
            labelFontSize=12,
            titleFontSize=12,
            grid=False
        )
    ),
    y=alt.Y('Total Enrollment:Q',
        axis=alt.Axis(
            title='Total Enrollment (millions)',
            labelExpr="datum.value/1000000",
            labelFontSize=12,
            titleFontSize=12,
            grid=True,
            gridOpacity=0.3
        ),
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color('Gender:N',
        scale=alt.Scale(domain=['Boys','Girls'], range=['#8B7BA8','#CD9368']),
        legend=alt.Legend(
            orient='top-left',
            titleFontSize=12,
            labelFontSize=12,
            symbolSize=200,
            symbolStrokeWidth=3
        )
    ),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Gender:N', title='Gender'),
        alt.Tooltip('Total Enrollment:Q', title='Enrollment', format=',.0f')
    ]
).properties(
    width=700,
    height=400,
    title={
        'text': 'Total School Enrollment Over Time by Gender',
        'subtitle': 'All India, Classes 1-12',
        'fontSize': 16,
        'subtitleFontSize': 12,
        'subtitleColor': '#666',
        'anchor': 'start'
    }
).configure_view(
    stroke='black',
    strokeWidth=1
).configure_axis(
    domainWidth=1
)

line_chart

alt.Chart(...)